# KARPHY'S CNN 
The notebook is used to experiment with video classification using karphy's CNN as discussed in https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42455.pdf

In [84]:
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow_datasets as tfds
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

## Reading in data
The "ucf101" dataset from tensorflow will be used. More information on this dataset can be found here URL: https://www.crcv.ucf.edu/data/UCF101.php

In [85]:
ucf101_dataset, ucf101_info = tfds.load(name="ucf101", with_info=True)
ucf101_train , ucf101_test = ucf101_dataset["train"], ucf101_dataset["test"]
print(type(ucf101_info))
assert isinstance(ucf101_train, tf.data.Dataset)
assert isinstance(ucf101_test, tf.data.Dataset)

print('number of training examples:', ucf101_info.splits["train"].num_examples)
print('number of test examples:', ucf101_info.splits["test"].num_examples)
print('number of labels:', ucf101_info.features["label"].num_classes)

<class 'tensorflow_datasets.core.dataset_info.DatasetInfo'>
number of training examples: 9537
number of test examples: 3783
number of labels: 101


### Looking into the dataset
The dataset has a pixel size of 256 x 256 x 3 for each video. 

In [76]:
print(ucf101_train.take(1))

<DatasetV1Adapter shapes: {video: (?, 256, 256, 3), label: ()}, types: {video: tf.uint8, label: tf.int64}>


### Formating data
* normalizing pixels? why again
* The pixels are rescaled to a size of 170 x 170 x 3 as suggested by the paper 
* The models suggested in the paper can take one frame of 10 frames depending if the model is early fusion of late fusion

In [77]:
IMG_SIZE = 170

def format_videos(dataset):
    dataset["video"] = tf.cast(dataset["video"], tf.float32)
    dataset["video"] = (dataset["video"]/255)
    dataset["video"] = tf.image.resize(dataset["video"], (IMG_SIZE,IMG_SIZE))
    return dataset
    

In [78]:
def select_first_frame(dataset):
    dataset["video"] = dataset["video"][0]
    return dataset 
    

In [79]:
def select_frame_from_i_to_T(dataset,i, T):
    dataset["video"] = dataset["video"][i:T]
    return dataset 

In [80]:
def select_frame_at_T(dataset, T):
    dataset["video"] = dataset["video"][T]
    return dataset 

In [81]:
def convert_to_tuple(dataset):
    x = dataset["video"]
    y = dataset["label"]
    return x,y 

changing resizing image and 

In [82]:
train = ucf101_train.map(format_videos)
train = train.map(select_first_frame)
train = train.map(convert_to_tuple)

test = ucf101_test.map(format_videos)
test = test.map(select_first_frame)
test = test.map(convert_to_tuple)


for d in ucf101_train.take(1):
    print(d["label"])
    print(d["video"].shape)  
for x, y in train.take(1):
    print(x.shape)   

tf.Tensor(18, shape=(), dtype=int64)
(126, 256, 256, 3)
(170, 170, 3)


Displaying images in video last image

loading test dataset

In [83]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 2 #1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([Dimension(32), Dimension(170), Dimension(170), Dimension(3)])

##

## Single Frame model

Creating single frame model from paper

Creating custom layer for local response normalization

In [57]:
class MyLRNLayer(tf.keras.layers.Layer):
    def __init__(self, depth_radius=5,bias=1,alpha=1,beta=0.5, **kwargs):
#         self.output_dim = output_dim
        self.depth_radius = depth_radius
        self.bias = bias
        self.alpha = alpha
        self.beta = beta
        super(MyLRNLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(None),
                                      initializer='uniform',
                                      trainable=False)
        super(MyLRNLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.nn.local_response_normalization(x,self.depth_radius,self.bias,self.alpha,self.beta)

#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.output_dim)


In [58]:
layer = MyLRNLayer()
print(layer(tf.zeros([1, 54, 54, 96])).shape)
print(len(layer.trainable_variables))

(1, 54, 54, 96)
0


In [59]:
modellrn = tf.keras.models.Sequential([MyLRNLayer(input_shape=(170, 170, 3),batch_size=1)])
modellrn.build()
modellrn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_lrn_layer_56 (MyLRNLayer) (1, 170, 170, 3)          1         
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________


In [60]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3)),
    
  MyLRNLayer(),  
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu'),
    
  MyLRNLayer(), 
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
    
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
    
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(101, activation='softmax')
])

In [61]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_139 (Conv2D)          (None, 54, 54, 96)        34944     
_________________________________________________________________
my_lrn_layer_57 (MyLRNLayer) (None, 54, 54, 96)        1         
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 23, 23, 256)       614656    
_________________________________________________________________
my_lrn_layer_58 (MyLRNLayer) (None, 23, 23, 256)       1         
_________________________________________________________________
conv2d_141 (Conv2D)          (None, 21, 21, 384)       885120    
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 10, 10, 384)      

In [62]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_batches, 
      epochs=1,
      verbose=1)

     52/Unknown - 3009s 58s/step - loss: 4.6234 - acc: 0.0120

# Improving the single frame model by using batch normalization 

In [65]:
batch_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3), batch_size=32),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu'),
    
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
    
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(101, activation='softmax')
])

In [66]:
batch_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_144 (Conv2D)          (32, 54, 54, 96)          34944     
_________________________________________________________________
batch_normalization_2 (Batch (32, 54, 54, 96)          384       
_________________________________________________________________
max_pooling2d_84 (MaxPooling (32, 27, 27, 96)          0         
_________________________________________________________________
conv2d_145 (Conv2D)          (32, 23, 23, 256)         614656    
_________________________________________________________________
batch_normalization_3 (Batch (32, 23, 23, 256)         1024      
_________________________________________________________________
conv2d_146 (Conv2D)          (32, 21, 21, 384)         885120    
_________________________________________________________________
max_pooling2d_85 (MaxPooling (32, 10, 10, 384)        

In [67]:
batch_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [69]:
batch_model.fit(train_batches, 
      epochs=3,
      verbose=1)

Epoch 1/3
      1/Unknown - 7s 7s/step - loss: 4.7061 - acc: 0.0000e+00

KeyboardInterrupt: 

In [22]:
data, info = tfds.load("ucf101", with_info=True)
train_data, test_data = data['train'], data['test']
assert isinstance(train_data, tf.data.Dataset)
print(info.features['label'].num_classes)
print(info.splits['train'].num_examples)

101
9537


In [ ]:
history = model.fit_generator(train_generator,
#       steps_per_epoch=8,  
      epochs=1,
      verbose=1,
      callbacks=[callbacks])

# Early Fusion



## Early Fusion with Local response normalization

In [1]:
train_multi_frame_T = ucf101_train.map(format_videos)
train_multi_frame_T = train_multi_frame_T.map(lambda x: select_frame_from_i_to_T(x,10,20))
train_multi_frame_T = train_multi_frame_T.map(convert_to_tuple)

test_multi_frame_T = ucf101_test.map(format_videos)
test_multi_frame_T = test_multi_frame_T.map(lambda x:select_frame_from_i_to_T(x,10,20))
test_multi_frame_T = test_multi_frame_T.map(convert_to_tuple)

train_multi_frame_T_batches = train_multi_frame_T.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_multi_frame_T_batches = test_multi_frame_T.batch(BATCH_SIZE)

for image_batch, label_batch in train_multi_frame_T_batches.take(1):
    pass

image_batch.shape


NameError: name 'ucf101_train' is not defined

In [70]:
model_early_fusion_lrn = tf.keras.models.Sequential([
  tf.keras.layers.Conv3D(96, (11,11,3), strides=3 , activation='relu', input_shape=(10,170, 170, 3)),
    
  MyLRNLayer(),  
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu'),
    
  MyLRNLayer(), 
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
    
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
    
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(101, activation='softmax')
])

ValueError: Negative dimension size caused by subtracting 11 from 10 for 'conv3d_1/Conv3D' (op: 'Conv3D') with input shapes: [?,10,170,170,3], [11,11,3,3,96].

In [ ]:
model_early_fusion_lrn.summary()

In [ ]:
model_early_fusion_lrn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [77]:
model_early_fusion_lrn.fit(train_multi_frame_T_batches, 
      epochs=3,
      verbose=1)

Epoch 1/3


ValueError: Error when checking input: expected conv2d_17_input to have 4 dimensions, but got array with shape (32, 10, 170, 170, 3)

## Early Fusion with Batch normalization

# Late Fusion

In [ ]:
x = model.layers[index].output

## Late Fusion with Local response normalization

In [28]:
train_clip_start = tf.keras.Input(shape=(170, 170, 3), name='start')  # Variable-length sequence of ints
train_clip_end = tf.keras.Input(shape=(170, 170, 3), name='end')  # Variable-length sequence of ints

# Embed each word in the title into a 64-dimensional vector
start_features = tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3))(train_clip_start)
start_features = MyLRNLayer()(start_features)  
start_features = tf.keras.layers.MaxPooling2D(2, 2)(start_features) 
start_features = tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu')(start_features) 
start_features = MyLRNLayer()(start_features) 
start_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.MaxPooling2D(2,2)(start_features)
start_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.MaxPooling2D(2,2)(start_features)  


# Embed each word in the title into a 64-dimensional vector
end_features = tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3))(train_clip_end)
end_features = MyLRNLayer()(end_features)  
end_features = tf.keras.layers.MaxPooling2D(2, 2)(end_features) 
end_features = tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu')(end_features) 
end_features = MyLRNLayer()(end_features) 
end_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.MaxPooling2D(2,2)(end_features)
end_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.MaxPooling2D(2,2)(end_features)  

# Merge all available features into a single large vector via concatenation
x = tf.keras.layers.concatenate([start_features, end_features])
x =  tf.keras.layers.Flatten()(x)
x =  tf.keras.layers.Dense(4096, activation='relu')(x)
x =  tf.keras.layers.Dense(4096, activation='relu')(x)
x =  tf.keras.layers.Dense(101, activation='softmax')(x)

# Instantiate an end-to-end model predicting both priority and department
late_fusion_model = tf.keras.Model(inputs=[train_clip_start, train_clip_end],
                    outputs=[x])
    
# # Embed each word in the text into a 64-dimensional vector
# body_features = layers.Embedding(num_words, 64)(body_input)

# # Reduce sequence of embedded words in the title into a single 128-dimensional vector
# title_features = layers.LSTM(128)(title_features)
# # Reduce sequence of embedded words in the body into a single 32-dimensional vector
# body_features = layers.LSTM(32)(body_features)

# # Merge all available features into a single large vector via concatenation
# x = layers.concatenate([title_features, body_features, tags_input])

# # Stick a logistic regression for priority prediction on top of the features
# priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(x)
# # Stick a department classifier on top of the features
# department_pred = layers.Dense(num_departments, activation='softmax', name='department')(x)

# # Instantiate an end-to-end model predicting both priority and department
# model = keras.Model(inputs=[title_input, body_input, tags_input],
#                     outputs=[priority_pred, department_pred])

In [29]:
late_fusion_model_LRN.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
start (InputLayer)              [(None, 170, 170, 3) 0                                            
__________________________________________________________________________________________________
end (InputLayer)                [(None, 170, 170, 3) 0                                            
__________________________________________________________________________________________________
conv2d_124 (Conv2D)             (None, 54, 54, 96)   34944       start[0][0]                      
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 54, 54, 96)   34944       end[0][0]                        
____________________________________________________________________________________________

## Late Fusion with Batch normalization

In [72]:
train_clip_start = tf.keras.Input(shape=(170, 170, 3), name='start')  # Variable-length sequence of ints
train_clip_end = tf.keras.Input(shape=(170, 170, 3), name='end')  # Variable-length sequence of ints

# Embed each word in the title into a 64-dimensional vector
start_features = tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3))(train_clip_start)
start_features = tf.keras.layers.BatchNormalization()(start_features)  
start_features = tf.keras.layers.MaxPooling2D(2, 2)(start_features) 
start_features = tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu')(start_features) 
start_features = tf.keras.layers.BatchNormalization()(start_features) 
start_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.MaxPooling2D(2,2)(start_features)
start_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu')(start_features)
start_features = tf.keras.layers.MaxPooling2D(2,2)(start_features)  


# Embed each word in the title into a 64-dimensional vector
end_features = tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3))(train_clip_end)
end_features = tf.keras.layers.BatchNormalization()(end_features)  
end_features = tf.keras.layers.MaxPooling2D(2, 2)(end_features) 
end_features = tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu')(end_features) 
end_features = tf.keras.layers.BatchNormalization()(end_features) 
end_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.MaxPooling2D(2,2)(end_features)
end_features = tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu')(end_features)
end_features = tf.keras.layers.MaxPooling2D(2,2)(end_features)  

# Merge all available features into a single large vector via concatenation
x = tf.keras.layers.concatenate([start_features, end_features])
x =  tf.keras.layers.Flatten()(x)
x =  tf.keras.layers.Dense(4096, activation='relu')(x)
x =  tf.keras.layers.Dense(4096, activation='relu')(x)
x =  tf.keras.layers.Dense(101, activation='softmax')(x)

# Instantiate an end-to-end model predicting both priority and department
late_fusion_model_bn = tf.keras.Model(inputs=[train_clip_start, train_clip_end],
                    outputs=[x])

In [73]:
late_fusion_model_bn.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
start (InputLayer)              [(None, 170, 170, 3) 0                                            
__________________________________________________________________________________________________
end (InputLayer)                [(None, 170, 170, 3) 0                                            
__________________________________________________________________________________________________
conv2d_163 (Conv2D)             (None, 54, 54, 96)   34944       start[0][0]                      
__________________________________________________________________________________________________
conv2d_168 (Conv2D)             (None, 54, 54, 96)   34944       end[0][0]                        
____________________________________________________________________________________________

# Slow Fusion

In [ ]:
Slow Fusion

In [ ]:
inputs_1 = tf.keras.Input(shape=(32,))  # Returns a placeholder tensor
inputs_T = tf.keras.Input(shape=(32,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

In [79]:
tf.keras.layers?

TypeError: 'MapDataset' object is not subscriptable

In [73]:
ucf101_train.map?

In [102]:
tf.keras.layers.Conv3D?